# App Rating
* Important note:
* Any transformation we do should be contained in a function so we can reuse it later

## Download data from kaggle

In [2]:
!uv add kaggle

Resolved 102 packages in 1.21s
Prepared 5 packages in 628ms
Installed 5 packages in 84ms
 + kaggle==1.7.4.2
 + protobuf==6.30.2
 + python-slugify==8.0.4
 + text-unidecode==1.3
 + tqdm==4.67.1


In [5]:
!kaggle competitions download -c app-rating-competition

In [6]:
import zipfile

with zipfile.ZipFile("app-rating-competition.zip", 'r') as zip_ref:
    zip_ref.extractall("app-rating-competition")

In [7]:
!uv add pandas

Resolved 106 packages in 1.19s
Prepared 4 packages in 4.46s
Installed 4 packages in 1.03s
 + numpy==2.2.5
 + pandas==2.2.3
 + pytz==2025.2
 + tzdata==2025.2


In [9]:
import pandas as pd

train_df = pd.read_csv("app-rating-competition/train.csv")
test_df = pd.read_csv("app-rating-competition/test.csv")
sample_submission_df = pd.read_csv("app-rating-competition/SampleSubmission.csv")

## Data Exploration

In [10]:
train_df.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,Y
0,Coloring book moana,ART_AND_DESIGN,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,3.9
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,4.7
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4.3
3,Paper flowers instructions,ART_AND_DESIGN,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,4.4
4,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,3.8


In [13]:
train_df

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,Y
0,Coloring book moana,ART_AND_DESIGN,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,3.9
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,4.7
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4.3
3,Paper flowers instructions,ART_AND_DESIGN,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,4.4
4,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8963,FR Forms,BUSINESS,0,9.6M,10+,Free,0,Everyone,Business,"September 29, 2016",1.1.5,4.0 and up,NaN
8964,Sya9a Maroc - FR,FAMILY,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up,4.5
8965,Fr. Mike Schmitz Audio Teachings,FAMILY,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up,5.0
8966,Parkinson Exercices FR,MEDICAL,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up,NaN


### Column Observation
* X0 → App Name (Names)
    * Has duplicates
* X1 → App Category (Categorical)
    * Most frequent is FAMILY (1605)
    * 34 distinct categories
* X2 → Size in bytes probably (Numerical)
    * Could be used for weighted rating or normalization
* X3 → Downloads in millions (Numerical)
    * Has 1359 entries "Varies with device"
    * We might have to edit this with the app name duplicates
* X4 → Number of reviews (Categorical)
* X5 → Free/Paid (Categorical)
    * Has one null we should explore before handling
* X6 → Price if paid (Numerical)
    * Check for any free with price or any paid with 0
* X7 → Age Rating (Categorical)
    * 6 categories
    * Has a null value we should check before handling
    * Top is Everyone (7222)
* X8 → App Tags (Categorical) (Should be enhanced)
    * Most frequent is Tools (705)
* X9 → Release Date (Will be heavily feature extracted)
    * Month/Day/Year
    * Maybe convert to days (released on weekend etc.)
    * Could find if released on specific holidays (Christmas, etc.)
* X10 → App Version (I think) (Numerical or Categorical)
    * 7 null values
    * Top is "Varies with device" 1173
* X11 → Compatible OS version (I think) (Categorical)
    * 3 Null values
    * Top is "4.1 and up" (2072)
* Y → App Rating
    * 1474 Null values (should be cleaned)
    * Has outliers heavily

### Rename Columns

In [14]:
def rename_columns(dataframe):
    reformated_df = dataframe.rename(
        columns={"X0": "app_name", "X1": "app_category", "X2": "size_bytes", "X3": "downloads_millions",
                 "X4": "reviews_count", "X5": "free_paid", "X6": "price_if_paid", "X7": "age_rating",
                 "X8": "app_tags", "X9": "release_date", "X10": "app_version",
                 "X11": "compatible_os_version"})
    return reformated_df

In [60]:
train_df1 = rename_columns(train_df)
train_df1

,app_name,app_category,size_bytes,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
0,Coloring book moana,ART_AND_DESIGN,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,3.9
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,4.7
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4.3
3,Paper flowers instructions,ART_AND_DESIGN,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,4.4
4,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8963,FR Forms,BUSINESS,0,9.6M,10+,Free,0,Everyone,Business,"September 29, 2016",1.1.5,4.0 and up,NaN
8964,Sya9a Maroc - FR,FAMILY,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up,4.5
8965,Fr. Mike Schmitz Audio Teachings,FAMILY,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up,5.0
8966,Parkinson Exercices FR,MEDICAL,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up,NaN


In [175]:
train_df_copy = train_df1.copy()

### Explore Null Values

#### Free/Paid nulls

In [176]:
train_df_copy[train_df_copy.free_paid.isnull()]

,app_name,app_category,size_bytes,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
7533,Command & Conquer: Rivals,FAMILY,0,Varies with device,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device,Varies with device,NaN


* This has size 0, non-specific downloads, zero reviews, and null Y, so it sounds safe to drop
* Out of curiosity let's check if any other apps exist with the same name

In [177]:
train_df_copy[train_df_copy['app_name'].str.contains("conquer", case=False, na=False)]

,app_name,app_category,size_bytes,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
7533,Command & Conquer: Rivals,FAMILY,0,Varies with device,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device,Varies with device,NaN


* So we will drop this

In [178]:
train_df_copy.dropna(subset=['free_paid'], inplace=True)  # drops column 7533

#### Age Rating Nulls

In [179]:
train_df_copy[train_df_copy.age_rating.isnull()]

,app_name,app_category,size_bytes,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
8653,Life Made WI-Fi Touchscreen Photo Frame,1.9,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN,19.0


* It looks like this is perfectly ok, but the values of columns are switched around
* We can check for all distinct values from now to find any similar cases

In [180]:
train_df_copy.loc[8653]

app_name                 Life Made WI-Fi Touchscreen Photo Frame
app_category                                                 1.9
size_bytes                                                  3.0M
downloads_millions                                        1,000+
reviews_count                                               Free
free_paid                                                      0
price_if_paid                                           Everyone
age_rating                                                   NaN
app_tags                                       February 11, 2018
release_date                                              1.0.19
app_version                                           4.0 and up
compatible_os_version                                        NaN
Y                                                           19.0
Name: 8653, dtype: object

##### Real Values
* app_category: compatible_os_version
* size_bytes: Y
* downloads_millions: size_bytes
* reviews_count: downloads_millions
* free_paid: reviews_count
* price_if_paid: free_paid
* age_rating: price_if_paid
* app_tags: age_rating
* release_date: app_tags
* app_version: release_date
* compatible_os_version: app_version
* Y: app_category

In [181]:
swap_pairs = [
    ("app_category", "compatible_os_version"),
    ("size_bytes", "Y"),
    ("downloads_millions", "size_bytes"),
    ("reviews_count", "downloads_millions"),
    ("free_paid", "reviews_count"),
    ("price_if_paid", "free_paid"),
    ("age_rating", "price_if_paid"),
    ("app_tags", "age_rating"),
    ("release_date", "app_tags"),
    ("app_version", "release_date"),
    ("compatible_os_version", "app_version"),
    ("Y", "app_category"),
]

# First read all col2 values
values_to_copy = {
    col1: train_df_copy.at[8653, col2]
    for col1, col2 in swap_pairs
}

# Then write them all at once
for col1, value in values_to_copy.items():
    train_df_copy.at[8653, col1] = value



C:\Users\yousi\AppData\Local\Temp\ipykernel_62072\582722994.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  train_df_copy.at[8653, col1] = value


In [182]:
train_df_copy.loc[8653]

app_name                 Life Made WI-Fi Touchscreen Photo Frame
app_category                                                 NaN
size_bytes                                                  19.0
downloads_millions                                          3.0M
reviews_count                                             1,000+
free_paid                                                   Free
price_if_paid                                                  0
age_rating                                              Everyone
app_tags                                                     NaN
release_date                                   February 11, 2018
app_version                                               1.0.19
compatible_os_version                                 4.0 and up
Y                                                            1.9
Name: 8653, dtype: object

* The row is still not fully clean
* Also I'm not sure 19.0 is possible in size as other sizes aren't in float
* And not sure rating is real
* Category and tags are null but they can be easily found online
* We will drop this for now but save it

In [183]:
dropped_row_8653 = train_df_copy.loc[[8653]]

train_df_copy.drop(8653, inplace=True)
dropped_row_8653

,app_name,app_category,size_bytes,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
8653,Life Made WI-Fi Touchscreen Photo Frame,NaN,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,1.9


#### App Version Nulls

In [184]:
train_df_copy[train_df_copy.app_version.isnull()]

,app_name,app_category,size_bytes,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
10,Learn To Draw Kawaii Characters,ART_AND_DESIGN,55,2.7M,"5,000+",Free,0,Everyone,Art & Design,"June 6, 2018",NaN,4.2 and up,3.2
1257,Market Update Helper,LIBRARIES_AND_DEMO,20145,11k,"1,000,000+",Free,0,Everyone,Libraries & Demo,"February 12, 2013",NaN,1.5 and up,4.1
5160,Virtual DJ Sound Mixer,TOOLS,4010,8.7M,"500,000+",Free,0,Everyone,Tools,"May 10, 2017",NaN,4.0 and up,4.2
5570,BT Master,FAMILY,0,222k,100+,Free,0,Everyone,Education,"November 6, 2016",NaN,1.6 and up,NaN
6082,Calculate My IQ,FAMILY,44,7.2M,"10,000+",Free,0,Everyone,Entertainment,"April 3, 2017",NaN,2.3 and up,NaN
6350,UFO-CQ,TOOLS,1,237k,10+,Paid,$0.99,Everyone,Tools,"July 4, 2016",NaN,2.0 and up,NaN
8541,La Fe de Jesus,BOOKS_AND_REFERENCE,8,658k,"1,000+",Free,0,Everyone,Books & Reference,"January 31, 2017",NaN,3.0 and up,NaN


* Turns out downloads are not only in millions, so let's edit the name, and we will solve this later
* I don't know what the size is in; it wouldn't make sense for it to be bytes, so let's check later
* Also notice the last four rows have null ratings as well
* But the rest have valid columns, so we could drop the last 4 for now and come up with app_version values for the rest

In [185]:
dropped_rows_null_version_rating = train_df_copy[
    train_df_copy[['app_version', 'Y']].isnull().all(axis=1)
]
dropped_rows_null_version_rating

,app_name,app_category,size_bytes,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
5570,BT Master,FAMILY,0,222k,100+,Free,0,Everyone,Education,"November 6, 2016",NaN,1.6 and up,NaN
6082,Calculate My IQ,FAMILY,44,7.2M,"10,000+",Free,0,Everyone,Entertainment,"April 3, 2017",NaN,2.3 and up,NaN
6350,UFO-CQ,TOOLS,1,237k,10+,Paid,$0.99,Everyone,Tools,"July 4, 2016",NaN,2.0 and up,NaN
8541,La Fe de Jesus,BOOKS_AND_REFERENCE,8,658k,"1,000+",Free,0,Everyone,Books & Reference,"January 31, 2017",NaN,3.0 and up,NaN


In [186]:
train_df_copy.drop(dropped_rows_null_version_rating.index, inplace=True)
train_df_copy[train_df_copy.app_version.isnull()]

,app_name,app_category,size_bytes,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
10,Learn To Draw Kawaii Characters,ART_AND_DESIGN,55,2.7M,"5,000+",Free,0,Everyone,Art & Design,"June 6, 2018",NaN,4.2 and up,3.2
1257,Market Update Helper,LIBRARIES_AND_DEMO,20145,11k,"1,000,000+",Free,0,Everyone,Libraries & Demo,"February 12, 2013",NaN,1.5 and up,4.1
5160,Virtual DJ Sound Mixer,TOOLS,4010,8.7M,"500,000+",Free,0,Everyone,Tools,"May 10, 2017",NaN,4.0 and up,4.2


* Now rename columns as stated

In [187]:
train_df_copy.rename(columns={
    'size_bytes': 'size_unknown_unit',
    'downloads_millions': 'downloads_unstandardized'
}, inplace=True)
train_df_copy.head()


,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
0,Coloring book moana,ART_AND_DESIGN,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,3.9
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,4.7
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4.3
3,Paper flowers instructions,ART_AND_DESIGN,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,4.4
4,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,3.8


#### Compatible OS Nulls

In [188]:
train_df_copy[train_df_copy.compatible_os_version.isnull()]

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
3599,[substratum] Vacuum: P,PERSONALIZATION,230,11M,"1,000+",Paid,$1.49,Everyone,Personalization,"July 20, 2018",4.4,NaN,4.4
3630,Pi Dark [substratum],PERSONALIZATION,189,2.1M,"10,000+",Free,0,Everyone,Personalization,"March 27, 2018",1.1,NaN,4.5


* substratum is a theme engine for android to allow users to set themes to their phone
* Pi Dark is 2MB in the play store, so the unit we have still doesn't make sense
* I'm not sure if we should drop these, so we'll leave them for now
* Let's look at any other substratum

In [189]:
train_df_copy[train_df_copy['app_name'].str.contains("substratum", case=False, na=False)]

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
3459,J Dark Ice Substratum Theme,PERSONALIZATION,164,33M,"10,000+",Free,0,Everyone,Personalization,"July 19, 2018",4.3,6.0 and up,4.5
3558,Dark Infusion Substratum Theme for Android N & O,PERSONALIZATION,133,59M,"1,000+",Paid,$2.49,Everyone,Personalization,"August 3, 2018",15.6,7.0 and up,4.8
3599,[substratum] Vacuum: P,PERSONALIZATION,230,11M,"1,000+",Paid,$1.49,Everyone,Personalization,"July 20, 2018",4.4,NaN,4.4
3630,Pi Dark [substratum],PERSONALIZATION,189,2.1M,"10,000+",Free,0,Everyone,Personalization,"March 27, 2018",1.1,NaN,4.5
3707,PitchBlack S - Samsung Substratum Theme “For O...,PERSONALIZATION,90,37M,"1,000+",Paid,$1.99,Everyone,Personalization,"August 4, 2018",12.6,7.0 and up,4.5
5442,[Substratum] M5 Theme,PERSONALIZATION,16,6.7M,"5,000+",Free,0,Everyone,Personalization,"March 10, 2018",1.3.4,7.0 and up,4.4


* All substratum themes have almost the same compatible_os_version as either 6.0 and up or 7.0 and up
* So we can fill our null values using this
* I will set both as 7.0 and up since it is most common

In [190]:
train_df_copy.fillna(value={"compatible_os_version": "7.0 and up"}, inplace=True)
train_df_copy[train_df_copy['app_name'].str.contains("substratum", case=False, na=False)]

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
3459,J Dark Ice Substratum Theme,PERSONALIZATION,164,33M,"10,000+",Free,0,Everyone,Personalization,"July 19, 2018",4.3,6.0 and up,4.5
3558,Dark Infusion Substratum Theme for Android N & O,PERSONALIZATION,133,59M,"1,000+",Paid,$2.49,Everyone,Personalization,"August 3, 2018",15.6,7.0 and up,4.8
3599,[substratum] Vacuum: P,PERSONALIZATION,230,11M,"1,000+",Paid,$1.49,Everyone,Personalization,"July 20, 2018",4.4,7.0 and up,4.4
3630,Pi Dark [substratum],PERSONALIZATION,189,2.1M,"10,000+",Free,0,Everyone,Personalization,"March 27, 2018",1.1,7.0 and up,4.5
3707,PitchBlack S - Samsung Substratum Theme “For O...,PERSONALIZATION,90,37M,"1,000+",Paid,$1.99,Everyone,Personalization,"August 4, 2018",12.6,7.0 and up,4.5
5442,[Substratum] M5 Theme,PERSONALIZATION,16,6.7M,"5,000+",Free,0,Everyone,Personalization,"March 10, 2018",1.3.4,7.0 and up,4.4


#### App Rating nulls

In [191]:
train_df_copy[train_df_copy.Y.isnull()]

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
16,Mcqueen Coloring pages,ART_AND_DESIGN,61,7.0M,"100,000+",Free,0,Everyone,Art & Design;Action & Adventure,"March 7, 2018",1.0.0,4.1 and up,NaN
79,Wrinkles and rejuvenation,BEAUTY,182,5.7M,"100,000+",Free,0,Everyone 10+,Beauty,"September 20, 2017",8.0,3.0 and up,NaN
87,Manicure - nail design,BEAUTY,119,3.7M,"50,000+",Free,0,Everyone,Beauty,"July 23, 2018",1.3,4.1 and up,NaN
90,Skin Care and Natural Beauty,BEAUTY,654,7.4M,"100,000+",Free,0,Teen,Beauty,"July 17, 2018",1.15,4.1 and up,NaN
93,"Secrets of beauty, youth and health",BEAUTY,77,2.9M,"10,000+",Free,0,Mature 17+,Beauty,"August 8, 2017",2.0,2.3 and up,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8954,Cardio-FR,MEDICAL,67,82M,"10,000+",Free,0,Everyone,Medical,"July 31, 2018",2.2.2,4.4 and up,NaN
8955,Naruto & Boruto FR,SOCIAL,7,7.7M,100+,Free,0,Teen,Social,"February 2, 2018",1.0,4.0 and up,NaN
8960,payermonstationnement.fr,MAPS_AND_NAVIGATION,38,9.8M,"5,000+",Free,0,Everyone,Maps & Navigation,"June 13, 2018",2.0.148.0,4.0 and up,NaN
8963,FR Forms,BUSINESS,0,9.6M,10+,Free,0,Everyone,Business,"September 29, 2016",1.1.5,4.0 and up,NaN


In [192]:
train_df_copy.Y.isnull().sum()

np.int64(1469)

* We have 1469 nulls which is too much to handle, so we can return to this later after more cleaning

### Explore Data Types
* Data types for a column should be consistent, so we should look for inconsistent rows and clean them

In [193]:
train_df_copy.dtypes

app_name                    object
app_category                object
size_unknown_unit           object
downloads_unstandardized    object
reviews_count               object
free_paid                   object
price_if_paid               object
age_rating                  object
app_tags                    object
release_date                object
app_version                 object
compatible_os_version       object
Y                           object
dtype: object

* Let's check for individual columns types as well as all unique values where possible

#### App Name

In [194]:
train_df_copy['app_name'].map(type).value_counts()


app_name
<class 'str'>    8962
Name: count, dtype: int64

In [195]:
train_df_copy['app_name'].value_counts()

app_name
ROBLOX                                               8
CBS Sports App - Scores, News, Stats & Watch Live    7
8 Ball Pool                                          6
Temple Run 2                                         6
Helix Jump                                           6
                                                    ..
FieldBi FR Offline                                   1
Golden Dictionary (FR-AR)                            1
FR: My Secret Pets!                                  1
DICT.fr Mobile                                       1
Fr Agnel Pune                                        1
Name: count, Length: 8154, dtype: int64

* Duplicated Names could cause a problem, we'll check later

#### App Category

In [196]:
train_df_copy['app_category'].map(type).value_counts()

app_category
<class 'str'>    8962
Name: count, dtype: int64

In [197]:
train_df_copy['app_category'].value_counts()

app_category
FAMILY                 1602
GAME                    929
TOOLS                   705
BUSINESS                403
MEDICAL                 395
PRODUCTIVITY            346
PERSONALIZATION         334
COMMUNICATION           334
SPORTS                  316
LIFESTYLE               315
FINANCE                 298
HEALTH_AND_FITNESS      287
PHOTOGRAPHY             278
SOCIAL                  251
NEWS_AND_MAGAZINES      249
SHOPPING                212
TRAVEL_AND_LOCAL        202
DATING                  192
BOOKS_AND_REFERENCE     187
VIDEO_PLAYERS           144
ENTERTAINMENT           126
EDUCATION               125
MAPS_AND_NAVIGATION     117
FOOD_AND_DRINK          102
HOUSE_AND_HOME           72
AUTO_AND_VEHICLES        71
LIBRARIES_AND_DEMO       70
WEATHER                  66
EVENTS                   56
ART_AND_DESIGN           49
PARENTING                44
COMICS                   43
BEAUTY                   42
Name: count, dtype: int64

#### Size

In [198]:
train_df_copy['size_unknown_unit'].map(type).value_counts()

size_unknown_unit
<class 'str'>    8962
Name: count, dtype: int64

In [199]:
train_df_copy['size_unknown_unit'].value_counts()

size_unknown_unit
0         594
1         262
2         192
3         161
4         118
         ... 
36815       1
87510       1
398307      1
455         1
1035        1
Name: count, Length: 5006, dtype: int64

* Apps with size 0 could cause a problem, so check later
* Also, size shouldn't be string unless we turn it categorical (<100, <10, etc.)

#### Downloads

In [200]:
train_df_copy['downloads_unstandardized'].map(type).value_counts()

downloads_unstandardized
<class 'str'>    8962
Name: count, dtype: int64

In [201]:
train_df_copy['downloads_unstandardized'].value_counts()

downloads_unstandardized
Varies with device    1358
12M                    174
11M                    157
14M                    157
13M                    152
                      ... 
404k                     1
420k                     1
992k                     1
73k                      1
518k                     1
Name: count, Length: 414, dtype: int64

* Varies with device could cause a problem, but we should turn this numerical.
* We only have numbers in millions (M) and thousands (K), so we should standardize

#### Reviews Number

In [202]:
train_df_copy['reviews_count'].map(type).value_counts()

reviews_count
<class 'str'>    8962
Name: count, dtype: int64

In [203]:
train_df_copy['reviews_count'].value_counts()

reviews_count
1,000,000+        1265
10,000,000+        991
100,000+           932
10,000+            868
1,000+             751
100+               656
5,000,000+         604
500,000+           447
5,000+             388
50,000+            386
10+                368
100,000,000+       337
500+               283
50,000,000+        229
50+                195
5+                  78
1+                  67
500,000,000+        57
1,000,000,000+      46
0+                  14
Name: count, dtype: int64

* Looks fine but we should try it as categorical and numerical

#### Free/Paid

In [204]:
train_df_copy['free_paid'].map(type).value_counts()

free_paid
<class 'str'>    8962
Name: count, dtype: int64

In [205]:
train_df_copy['free_paid'].value_counts()

free_paid
Free    8298
Paid     664
Name: count, dtype: int64

* Looks fine and will be categorical
* Let's look at any incorrect mappings (free but has price, paid but price 0

#### Price if paid

In [206]:
train_df_copy['price_if_paid'].map(type).value_counts()

price_if_paid
<class 'str'>    8962
Name: count, dtype: int64

In [207]:
train_df_copy['price_if_paid'].value_counts()

price_if_paid
0          8298
$0.99       124
$2.99        99
$1.99        64
$4.99        60
           ... 
$3.61         1
$394.99       1
$1.26         1
$1.20         1
$1.04         1
Name: count, Length: 86, dtype: int64

In [208]:
inconsistent_price_rows = train_df_copy[
    ((train_df_copy['free_paid'] == "Free") & (train_df_copy['price_if_paid'] != "0")) |
    ((train_df_copy['free_paid'] == "Paid") & (
                (train_df_copy['price_if_paid'] == "0") | (train_df_copy['price_if_paid'].isnull())))
    ]
inconsistent_price_rows

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y


* Looks fine, we should remove the dollar sign to make it numerical
* Maybe try price tiers categories
* We also have no inconsistent prices

#### Age Rating

In [209]:
train_df_copy['age_rating'].map(type).value_counts()

age_rating
<class 'str'>    8962
Name: count, dtype: int64

In [210]:
train_df_copy['age_rating'].value_counts()

age_rating
Everyone           7218
Teen               1000
Mature 17+          406
Everyone 10+        333
Adults only 18+       3
Unrated               2
Name: count, dtype: int64

* Looks fine maybe clean up the string

#### App Tags

In [211]:
train_df_copy['app_tags'].map(type).value_counts()

app_tags
<class 'str'>    8962
Name: count, dtype: int64

In [212]:
train_df_copy['app_tags'].value_counts()

app_tags
Tools                       704
Entertainment               528
Education                   453
Business                    403
Medical                     395
                           ... 
Role Playing;Brain Games      1
Racing;Pretend Play           1
Puzzle;Creativity             1
Communication;Creativity      1
Strategy;Creativity           1
Name: count, Length: 114, dtype: int64

* This is currently too much and will need cleaning

#### Release Date

In [213]:
train_df_copy['release_date'].map(type).value_counts()

release_date
<class 'str'>    8962
Name: count, dtype: int64

In [214]:
train_df_copy['release_date'].value_counts()

release_date
August 3, 2018       249
August 2, 2018       248
July 31, 2018        237
August 1, 2018       230
July 30, 2018        170
                    ... 
March 7, 2014          1
December 18, 2013      1
February 27, 2015      1
January 29, 2016       1
June 17, 2012          1
Name: count, Length: 1299, dtype: int64

In [215]:
expected_format = "%B %d, %Y"

invalid_dates = pd.to_datetime(
    train_df_copy['release_date'],
    format=expected_format,
    errors='coerce'  # Converts invalid formats to NaT (null)
).isna()

invalid_date_rows = train_df_copy[invalid_dates]
invalid_date_rows

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y


* All date rows are consistent in format
* However, this current release date will not help and will need to be extracted to more useful features

#### App Version

In [216]:
train_df_copy['app_version'].map(type).value_counts()

app_version
<class 'str'>      8959
<class 'float'>       3
Name: count, dtype: int64

In [217]:
train_df_copy['app_version'].value_counts()

app_version
Varies with device    1172
1.0                    714
1.1                    220
1.2                    149
2.0                    126
                      ... 
3.36                     1
Android 3.0 - 2017       1
2.1.9f7                  1
4.6.1404                 1
4.7.0.2200000            1
Name: count, Length: 2502, dtype: int64

* Again varies with device is causing a problem here
* And we want to standardize the type here

In [218]:
train_df_copy[train_df_copy['app_version'].map(type) == float]

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
10,Learn To Draw Kawaii Characters,ART_AND_DESIGN,55,2.7M,"5,000+",Free,0,Everyone,Art & Design,"June 6, 2018",NaN,4.2 and up,3.2
1257,Market Update Helper,LIBRARIES_AND_DEMO,20145,11k,"1,000,000+",Free,0,Everyone,Libraries & Demo,"February 12, 2013",NaN,1.5 and up,4.1
5160,Virtual DJ Sound Mixer,TOOLS,4010,8.7M,"500,000+",Free,0,Everyone,Tools,"May 10, 2017",NaN,4.0 and up,4.2


In [219]:
dropped_rows_null_version = train_df_copy[
    train_df_copy[['app_version']].isnull().all(axis=1)
]
dropped_rows_null_version

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
10,Learn To Draw Kawaii Characters,ART_AND_DESIGN,55,2.7M,"5,000+",Free,0,Everyone,Art & Design,"June 6, 2018",NaN,4.2 and up,3.2
1257,Market Update Helper,LIBRARIES_AND_DEMO,20145,11k,"1,000,000+",Free,0,Everyone,Libraries & Demo,"February 12, 2013",NaN,1.5 and up,4.1
5160,Virtual DJ Sound Mixer,TOOLS,4010,8.7M,"500,000+",Free,0,Everyone,Tools,"May 10, 2017",NaN,4.0 and up,4.2


In [220]:
train_df_copy.dropna(subset=['app_version'], inplace=True)

* We'll drop these rows for now

#### Compatible OS

In [221]:
train_df_copy['compatible_os_version'].map(type).value_counts()

compatible_os_version
<class 'str'>    8959
Name: count, dtype: int64

In [222]:
train_df_copy['compatible_os_version'].value_counts()

compatible_os_version
4.1 and up            2072
4.0.3 and up          1228
4.0 and up            1124
Varies with device    1087
4.4 and up             817
2.3 and up             515
5.0 and up             508
4.2 and up             329
2.3.3 and up           234
2.2 and up             206
3.0 and up             203
4.3 and up             201
2.1 and up             113
1.6 and up              93
6.0 and up              54
7.0 and up              38
3.2 and up              34
2.0 and up              24
5.1 and up              18
1.5 and up              16
4.4W and up             12
3.1 and up              10
2.0.1 and up             6
8.0 and up               5
7.1 and up               3
4.0.3 - 7.1.1            2
7.0 - 7.1.1              1
5.0 - 8.0                1
4.1 - 7.1.1              1
5.0 - 6.0                1
1.0 and up               1
2.2 - 7.1.1              1
5.0 - 7.1.1              1
Name: count, dtype: int64

* Again varies with device
* Also some fields have "5.0 and up" format and others have "5.0 - 7.1.1" format, we should explore these formats

#### App Rating

In [223]:
train_df_copy['Y'].map(type).value_counts()

Y
<class 'float'>    8959
Name: count, dtype: int64

In [224]:
train_df_copy['Y'].value_counts()

Y
4.4    891
4.3    881
4.5    840
4.2    748
4.6    658
4.1    575
4.0    439
4.7    385
3.9    304
3.8    244
5.0    207
3.7    196
4.8    177
3.6    143
3.5    131
3.4    104
3.3     83
4.9     74
3.0     62
3.1     55
3.2     53
2.8     36
2.9     34
2.6     24
2.7     21
2.5     18
2.4     15
1.0     14
2.3     13
1.9     12
2.2     12
2.0     11
2.1      8
1.8      7
1.7      6
1.6      4
1.4      3
1.2      1
1.5      1
Name: count, dtype: int64

### Handling Duplicates

#### Duplicate Names

In [225]:
train_df_copy['app_name'].value_counts()

app_name
ROBLOX                                                8
CBS Sports App - Scores, News, Stats & Watch Live     7
Temple Run 2                                          6
slither.io                                            6
Zombie Catchers                                       6
                                                     ..
FR Forms                                              1
Sya9a Maroc - FR                                      1
Fr. Mike Schmitz Audio Teachings                      1
Parkinson Exercices FR                                1
U Launcher Lite – FREE Live Cool Themes, Hide Apps    1
Name: count, Length: 8151, dtype: int64

In [226]:
roblox_rows = train_df_copy[train_df_copy['app_name'] == "ROBLOX"]
roblox_rows

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
1334,ROBLOX,GAME,4447388,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5
1370,ROBLOX,GAME,4447346,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5
1409,ROBLOX,GAME,4448791,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5
1480,ROBLOX,GAME,4449882,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5
1502,ROBLOX,GAME,4449910,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5
1627,ROBLOX,FAMILY,4449910,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5
1780,ROBLOX,FAMILY,4450890,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5
3663,ROBLOX,FAMILY,4443407,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5


* These are duplicates, we will turn them to 1 and use the GAME tag as its the median
* We will get the mean size as it's different

In [227]:
app_category_median = roblox_rows['app_category'].mode()[0]

app_category_median

'GAME'

In [228]:
size_mean = pd.to_numeric(roblox_rows['size_unknown_unit'], errors='coerce').mean()

size_mean

np.float64(4448440.5)

In [229]:
def drop_duplicates_specific_value(dataframe, column_name, value_name):
    is_column = dataframe[column_name] == value_name
    column_deduped = dataframe[is_column].drop_duplicates(subset=column_name, keep='first')

    non_column = dataframe[~is_column]

    return pd.concat([non_column, column_deduped], ignore_index=True)


In [232]:
train_df_copy = drop_duplicates_specific_value(train_df_copy, 'app_name', 'ROBLOX')
train_df_copy[train_df_copy['app_name'] == "ROBLOX"]

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
8951,ROBLOX,GAME,4447388,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5


In [233]:
import math

roblox_size_floored_str = str(math.floor(size_mean))
roblox_size_floored_str

'4448440'

In [234]:
train_df_copy.loc[train_df_copy['app_name'] == "ROBLOX", 'size_unknown_unit'] = roblox_size_floored_str
train_df_copy[train_df_copy['app_name'] == "ROBLOX"]

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
8951,ROBLOX,GAME,4448440,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",2.347.225742,4.1 and up,4.5


* Let's now handle other duplicates

In [235]:
train_df_copy['app_name'].value_counts()

app_name
CBS Sports App - Scores, News, Stats & Watch Live     7
slither.io                                            6
Temple Run 2                                          6
8 Ball Pool                                           6
Helix Jump                                            6
                                                     ..
Sya9a Maroc - FR                                      1
Fr. Mike Schmitz Audio Teachings                      1
Parkinson Exercices FR                                1
iHoroscope - 2018 Daily Horoscope & Astrology         1
U Launcher Lite – FREE Live Cool Themes, Hide Apps    1
Name: count, Length: 8151, dtype: int64

#### Drop perfect duplicates
* Let's drop perfect duplicates first (all columns equal)

In [239]:
train_df_copy_checkpoint = train_df_copy.copy()

In [240]:
train_df_copy.drop_duplicates(inplace=True)
train_df_copy['app_name'].value_counts()

app_name
Helix Jump                                            6
Zombie Catchers                                       6
8 Ball Pool                                           6
Angry Birds Classic                                   5
Temple Run 2                                          5
                                                     ..
Sya9a Maroc - FR                                      1
Fr. Mike Schmitz Audio Teachings                      1
Parkinson Exercices FR                                1
iHoroscope - 2018 Daily Horoscope & Astrology         1
U Launcher Lite – FREE Live Cool Themes, Hide Apps    1
Name: count, Length: 8151, dtype: int64

#### Duplicates w/ different sizes
* We still have duplicates, so we will handle them in different ways
* Let's first check for duplicates with different sizes only

In [245]:
def find_near_duplicates(df, except_col):
    columns_except = df.columns.difference([except_col])
    near_duplicates = df[df.duplicated(subset=columns_except, keep=False)]
    return near_duplicates


# Define columns to check for duplicates (exclude 'size_unknown_unit')
columns_except_size = train_df_copy.columns.difference(['size_unknown_unit'])

# Find duplicates based on all columns except size
near_duplicates = find_near_duplicates(train_df_copy, except_col='size_unknown_unit')
near_duplicates

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
28,Textgram - write on photos,ART_AND_DESIGN,295221,Varies with device,"10,000,000+",Free,0,Everyone,Art & Design,"July 30, 2018",Varies with device,Varies with device,4.4
104,Amazon Kindle,BOOKS_AND_REFERENCE,814080,Varies with device,"100,000,000+",Free,0,Teen,Books & Reference,"July 27, 2018",Varies with device,Varies with device,4.2
117,Spanish English Translator,BOOKS_AND_REFERENCE,87873,Varies with device,"10,000,000+",Free,0,Teen,Books & Reference,"May 28, 2018",Varies with device,Varies with device,4.2
122,NOOK App for NOOK Devices,BOOKS_AND_REFERENCE,19080,Varies with device,"500,000+",Free,0,Everyone,Books & Reference,"April 25, 2018",Varies with device,Varies with device,4.7
142,English Dictionary - Offline,BOOKS_AND_REFERENCE,341157,30M,"10,000,000+",Free,0,Everyone 10+,Books & Reference,"March 20, 2018",3.9.1,4.2 and up,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8420,Who Viewed My Facebook Profile - Stalkers Visi...,SOCIAL,273244,9.9M,"5,000,000+",Free,0,Everyone,Social,"June 24, 2018",4.1.1,4.0.3 and up,4.6
8784,Podcast App: Free & Offline Podcasts by Player FM,NEWS_AND_MAGAZINES,66407,19M,"1,000,000+",Free,0,Teen,News & Magazines,"July 25, 2018",4.1.0.72,4.0 and up,4.6
8843,FarmersOnly Dating,DATING,1145,1.4M,"100,000+",Free,0,Mature 17+,Dating,"February 25, 2016",2.2,4.0 and up,3.0
8846,Firefox Focus: The privacy browser,COMMUNICATION,36981,4.0M,"1,000,000+",Free,0,Everyone,Communication,"July 6, 2018",5.2,5.0 and up,4.4


* We will now take the mean size and drop duplicates and use the floored mean
* This is a helper function that we can use later

In [246]:
def deduplicate_with_aggregation(dataframe, group_by_except_col, target_col, strategy, key_col):
    """
    Deduplicates rows that are equal in all columns except one,
    aggregates the target column using a given strategy, and
    keeps only one row per duplicate group with the updated value.

    Parameters:
    - dataframe: pd.DataFrame
    - group_by_except_col: str → the column to ignore when checking for duplicates
    - target_col: str → the column to aggregate and update (usually same as group_by_except_col)
    - strategy: str → 'mean', 'median', or 'mode'
    - key_col: str → the column to identify duplicate groups (default 'app_name')

    Returns:
    - Cleaned DataFrame with duplicates dropped and target_col updated.
    """

    group_cols = dataframe.columns.difference([group_by_except_col])
    duplicate_rows = dataframe[dataframe.duplicated(subset=group_cols, keep=False)]
    grouped = duplicate_rows.groupby(list(group_cols))

    for _, group_df in grouped:
        # Aggregate value based on strategy
        values = group_df[target_col].dropna()

        if strategy == 'mean':
            result = pd.to_numeric(values, errors='coerce').mean()
            aggregated_value = str(math.floor(result)) if pd.notna(result) else None

        elif strategy == 'median':
            result = pd.to_numeric(values, errors='coerce').median()
            aggregated_value = str(math.floor(result)) if pd.notna(result) else None

        elif strategy == 'mode':
            mode_vals = values.mode()
            aggregated_value = str(mode_vals[0]) if not mode_vals.empty else None

        else:
            raise ValueError(f"Unsupported strategy: {strategy}")

        # Representative key value for filtering
        key_value = group_df[key_col].iloc[0]

        # Drop duplicates of this key
        dataframe = drop_duplicates_specific_value(dataframe, key_col, key_value)

        # Assign new value if available
        if aggregated_value is not None:
            dataframe.loc[dataframe[key_col] == key_value, target_col] = aggregated_value

    return dataframe


In [249]:
train_df_copy = deduplicate_with_aggregation(
    train_df_copy,
    group_by_except_col='size_unknown_unit',
    target_col='size_unknown_unit',
    strategy='mean',
    key_col='app_name'
)

find_near_duplicates(train_df_copy, except_col='size_unknown_unit')

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y


#### Duplicates w/ different categories

* Let's first check if this has helped any of our other cases

In [253]:
train_df_copy.Y.isnull().sum()

np.int64(1460)

In [255]:
(train_df_copy['compatible_os_version'] == "Varies with device").sum()

np.int64(859)

In [256]:
(train_df_copy['app_version'] == "Varies with device").sum()

np.int64(920)

In [258]:
(train_df_copy['downloads_unstandardized'] == "Varies with device").sum()

np.int64(1062)

* Null Y: 1469 -> 1460
* OS varies: 1087 -> 859
* App version varies: 1172 -> 920
* downloads varies: 1358 -> 1062
* Let's now find duplicates with different category

In [259]:
find_near_duplicates(train_df_copy, except_col='app_category')

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
467,Learn C++,EDUCATION,73404,5.3M,"1,000,000+",Free,0,Everyone,Education,"December 25, 2017",4.5.2,4.0 and up,4.6
509,Fuzzy Numbers: Pre-K Number Foundation,EDUCATION,21,44M,"1,000+",Paid,$5.99,Everyone,Education;Education,"July 21, 2017",1.3,4.1 and up,4.7
1127,Candy Bomb,GAME,42145,20M,"10,000,000+",Free,0,Everyone,Casual;Brain Games,"July 4, 2018",2.9.3181,4.0.3 and up,4.4
1140,Candy Bomb,FAMILY,42145,20M,"10,000,000+",Free,0,Everyone,Casual;Brain Games,"July 4, 2018",2.9.3181,4.0.3 and up,4.4
2362,Learn C++,FAMILY,73404,5.3M,"1,000,000+",Free,0,Everyone,Education,"December 25, 2017",4.5.2,4.0 and up,4.6
2563,Fuzzy Numbers: Pre-K Number Foundation,FAMILY,21,44M,"1,000+",Paid,$5.99,Everyone,Education;Education,"July 21, 2017",1.3,4.1 and up,4.7


* Using a median wouldn't apply here as we only have two values for each
* However, looking at the tags and name we can decide which category is most suitable
* Learn C++: EDUCATION
* Fuzzy Numbers: EDUCATION
* Candy Bomb: GAME

In [261]:
train_df_copy.loc[train_df_copy['app_name'] == "Learn C++", 'app_category'] = "EDUCATION"
train_df_copy.loc[train_df_copy['app_name'] == "Fuzzy Numbers: Pre-K Number Foundation", 'app_category'] = "EDUCATION"
train_df_copy.loc[train_df_copy['app_name'] == "Candy Bomb", 'app_category'] = "GAME"
train_df_copy.drop_duplicates(inplace=True)
find_near_duplicates(train_df_copy, except_col='app_category')

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y


#### Duplicates w/ different release date
* Only other near duplicates are release date

In [267]:
find_near_duplicates(train_df_copy, except_col='release_date')

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
1573,Target - now with Cartwheel,SHOPPING,68406,24M,"10,000,000+",Free,0,Everyone,Shopping,"July 25, 2018",6.25.0+1906001476,5.0 and up,4.1
1621,Target - now with Cartwheel,SHOPPING,68406,24M,"10,000,000+",Free,0,Everyone,Shopping,"August 6, 2018",6.25.0+1906001476,5.0 and up,4.1
1922,osmino Wi-Fi: free WiFi,TOOLS,134203,4.1M,"10,000,000+",Free,0,Everyone,Tools,"August 6, 2018",6.06.14,4.4 and up,4.2
7579,osmino Wi-Fi: free WiFi,TOOLS,134203,4.1M,"10,000,000+",Free,0,Everyone,Tools,"August 7, 2018",6.06.14,4.4 and up,4.2


* Again we will set this manually
* osmino Wi-Fi: free WiFi: August 6, 2018
* Target - now with Cartwheel:  July 25, 2018

In [272]:
train_df_copy.loc[train_df_copy['app_name'] == "Target - now with Cartwheel", 'release_date'] = "July 25, 2018"
train_df_copy.loc[train_df_copy['app_name'] == "osmino Wi-Fi: free WiFi", 'release_date'] = "August 6, 2018"
train_df_copy.drop_duplicates(inplace=True)
find_near_duplicates(train_df_copy, except_col='release_date')

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y


#### Duplicate Names Check

In [274]:
train_df_copy['app_name'].value_counts()

app_name
Video Editor                                          3
Solitaire                                             3
Angry Birds 2                                         2
AccuWeather: Daily Forecast & Live Weather Reports    2
Flipkart Online Shopping App                          2
                                                     ..
ASOS                                                  1
Booking.com Travel Deals                              1
Bowmasters                                            1
Wordscapes                                            1
U Launcher Lite – FREE Live Cool Themes, Hide Apps    1
Name: count, Length: 8151, dtype: int64

In [275]:
app_names_with_3 = train_df_copy['app_name'].value_counts()
app_names_with_3 = app_names_with_3[app_names_with_3 == 3].index
rows_with_3_duplicates = train_df_copy[train_df_copy['app_name'].isin(app_names_with_3)]
rows_with_3_duplicates

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
1024,Solitaire,GAME,254258,23M,"10,000,000+",Free,0,Everyone,Card,"August 1, 2018",2.137.0,4.1 and up,4.7
1124,Solitaire,GAME,154264,12M,"10,000,000+",Free,0,Everyone,Card,"June 8, 2018",Varies with device,Varies with device,4.7
1146,Solitaire,FAMILY,685,26M,"100,000+",Free,0,Everyone,Card;Brain Games,"July 16, 2018",1.7.101,4.1 and up,4.4
1273,Video Editor,FAMILY,159619,23M,"5,000,000+",Free,0,Everyone,Video Players & Editors;Creativity,"July 17, 2018",6.14.091,4.3 and up,4.1
2175,Video Editor,VIDEO_PLAYERS,159622,23M,"5,000,000+",Free,0,Everyone,Video Players & Editors;Creativity,"July 17, 2018",6.14.091,4.3 and up,4.1
3745,Video Editor,VIDEO_PLAYERS,1156,23M,"100,000+",Free,0,Everyone,Video Players & Editors,"October 20, 2015",1.3.9b,4.3 and up,3.8


* Most of these make sense other than the video editor which has 2 duplicates from the 3

In [276]:
train_df_copy.loc[
    (train_df_copy['app_name'] == "Video Editor") &
    (train_df_copy['reviews_count'] == "5,000,000+"),
    'app_category'
] = "VIDEO_PLAYERS"

rows_with_3_duplicates = train_df_copy[train_df_copy['app_name'].isin(app_names_with_3)]
rows_with_3_duplicates

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
1024,Solitaire,GAME,254258,23M,"10,000,000+",Free,0,Everyone,Card,"August 1, 2018",2.137.0,4.1 and up,4.7
1124,Solitaire,GAME,154264,12M,"10,000,000+",Free,0,Everyone,Card,"June 8, 2018",Varies with device,Varies with device,4.7
1146,Solitaire,FAMILY,685,26M,"100,000+",Free,0,Everyone,Card;Brain Games,"July 16, 2018",1.7.101,4.1 and up,4.4
1273,Video Editor,VIDEO_PLAYERS,159619,23M,"5,000,000+",Free,0,Everyone,Video Players & Editors;Creativity,"July 17, 2018",6.14.091,4.3 and up,4.1
2175,Video Editor,VIDEO_PLAYERS,159622,23M,"5,000,000+",Free,0,Everyone,Video Players & Editors;Creativity,"July 17, 2018",6.14.091,4.3 and up,4.1
3745,Video Editor,VIDEO_PLAYERS,1156,23M,"100,000+",Free,0,Everyone,Video Players & Editors,"October 20, 2015",1.3.9b,4.3 and up,3.8


In [278]:
train_df_copy.loc[
    (train_df_copy['app_name'] == "Video Editor") &
    (train_df_copy['reviews_count'] == "5,000,000+"),
    'size_unknown_unit'
] = "159620"

In [279]:
train_df_copy.drop_duplicates(inplace=True)
rows_with_3_duplicates = train_df_copy[train_df_copy['app_name'].isin(app_names_with_3)]
rows_with_3_duplicates

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
1024,Solitaire,GAME,254258,23M,"10,000,000+",Free,0,Everyone,Card,"August 1, 2018",2.137.0,4.1 and up,4.7
1124,Solitaire,GAME,154264,12M,"10,000,000+",Free,0,Everyone,Card,"June 8, 2018",Varies with device,Varies with device,4.7
1146,Solitaire,FAMILY,685,26M,"100,000+",Free,0,Everyone,Card;Brain Games,"July 16, 2018",1.7.101,4.1 and up,4.4
1273,Video Editor,VIDEO_PLAYERS,159620,23M,"5,000,000+",Free,0,Everyone,Video Players & Editors;Creativity,"July 17, 2018",6.14.091,4.3 and up,4.1
3745,Video Editor,VIDEO_PLAYERS,1156,23M,"100,000+",Free,0,Everyone,Video Players & Editors,"October 20, 2015",1.3.9b,4.3 and up,3.8


* Duplicated names with 2 are too much so we will have to use other ways

#### Duplicates w/ different sizes & categories

In [281]:
def find_almost_near_duplicates(df, except_cols):
    """
    Finds near-duplicate rows in a DataFrame, ignoring specified columns.

    Parameters:
    - df: pd.DataFrame
    - except_cols: list of str → columns to exclude when checking for duplicates

    Returns:
    - DataFrame containing near-duplicate rows
    """
    columns_except = df.columns.difference(except_cols)
    near_duplicates = df[df.duplicated(subset=columns_except, keep=False)]
    return near_duplicates


In [282]:
near_duplicates = find_almost_near_duplicates(train_df_copy, except_cols=['size_unknown_unit', 'app_category'])
near_duplicates

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
0,Coloring book moana,ART_AND_DESIGN,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,3.9
15,Mcqueen Coloring pages,ART_AND_DESIGN,61,7.0M,"100,000+",Free,0,Everyone,Art & Design;Action & Adventure,"March 7, 2018",1.0.0,4.1 and up,NaN
24,UNICORN - Color By Number & Pixel Art Coloring,ART_AND_DESIGN,8145,24M,"500,000+",Free,0,Everyone,Art & Design;Creativity,"August 2, 2018",1.0.9,4.4 and up,4.7
424,TED,EDUCATION,181893,18M,"10,000,000+",Free,0,Everyone 10+,Education,"July 27, 2018",3.2.5,4.1 and up,4.6
439,Monster Truck Driver & Racing,EDUCATION,748,51M,"1,000,000+",Free,0,Everyone,Education;Action & Adventure,"January 19, 2017",1.0.9,2.3 and up,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5919,Idle Heroes,FAMILY,417197,99M,"10,000,000+",Free,0,Everyone 10+,Role Playing,"May 30, 2018",1.14.0,2.3 and up,4.7
6847,Game for KIDS: KIDS match'em,FAMILY,7006,6.9M,"1,000,000+",Free,0,Everyone,Education,"October 23, 2016",1.17,2.3 and up,4.4
6931,Episode - Choose Your Story,FAMILY,1842381,Varies with device,"50,000,000+",Free,0,Teen,Simulation,"July 31, 2018",Varies with device,Varies with device,4.3
7005,The NBC App - Watch Live TV and Full Episodes,FAMILY,58104,17M,"5,000,000+",Free,0,Teen,Entertainment,"July 5, 2018",4.18.1,4.1 and up,4.1


* We have 64 of them, however we can notice that some fields have no download data
* We can drop these for now and think how we can use them later

##### Drop rows with all varies

In [285]:
dropped_rows_varies_downloads_version_os = near_duplicates[
    (near_duplicates['downloads_unstandardized'] == "Varies with device") &
    (near_duplicates['app_version'] == "Varies with device") &
    (near_duplicates['compatible_os_version'] == "Varies with device")
    ]

In [286]:
train_df_copy = train_df_copy.drop(index=dropped_rows_varies_downloads_version_os.index)

In [287]:
near_duplicates = find_almost_near_duplicates(train_df_copy, except_cols=['size_unknown_unit', 'app_category'])
near_duplicates

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
0,Coloring book moana,ART_AND_DESIGN,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,3.9
15,Mcqueen Coloring pages,ART_AND_DESIGN,61,7.0M,"100,000+",Free,0,Everyone,Art & Design;Action & Adventure,"March 7, 2018",1.0.0,4.1 and up,NaN
24,UNICORN - Color By Number & Pixel Art Coloring,ART_AND_DESIGN,8145,24M,"500,000+",Free,0,Everyone,Art & Design;Creativity,"August 2, 2018",1.0.9,4.4 and up,4.7
424,TED,EDUCATION,181893,18M,"10,000,000+",Free,0,Everyone 10+,Education,"July 27, 2018",3.2.5,4.1 and up,4.6
439,Monster Truck Driver & Racing,EDUCATION,748,51M,"1,000,000+",Free,0,Everyone,Education;Action & Adventure,"January 19, 2017",1.0.9,2.3 and up,4.4
445,Game for KIDS: KIDS match'em,EDUCATION,7005,6.9M,"1,000,000+",Free,0,Everyone,Education,"October 23, 2016",1.17,2.3 and up,4.4
510,Toca Life: City,EDUCATION,31085,24M,"500,000+",Paid,$3.99,Everyone,Education;Pretend Play,"July 6, 2018",1.5-play,4.4 and up,4.7
532,No.Draw - Colors by Number 2018,ENTERTAINMENT,235496,13M,"10,000,000+",Free,0,Everyone,Entertainment;Brain Games,"July 29, 2018",1.2.0,4.1 and up,4.5
559,Nick,ENTERTAINMENT,123279,25M,"10,000,000+",Free,0,Everyone 10+,Entertainment;Music & Video,"January 24, 2018",2.0.8,4.4 and up,4.2
561,The CW,ENTERTAINMENT,288150,21M,"5,000,000+",Free,0,Teen,Entertainment,"July 23, 2018",2.17,4.4 and up,4.4


In [305]:
def deduplicate_with_mean_and_tag_based_category(df, key_col='app_name'):
    """
    Deduplicates rows that are identical in all columns except for
    'size_unknown_unit' and 'app_category'. Uses:

    - Floored mean for size
    - Original logic: if exactly one category is found in app_tags → keep that
    - If not, use category_tag_map to infer from tags
    - If still ambiguous or no match, keep all rows

    Parameters:
    - df: DataFrame to clean
    - key_col: Column to identify unique groups (e.g., 'app_name')

    Returns:
    - Cleaned DataFrame with resolved duplicates
    """

    # Category-to-tag mapping
    category_tag_map = {
        "ART_AND_DESIGN": ["art & design"],
        "GAME": ["simulation", "action", "role playing", "puzzle"],
        "FAMILY": ["casual"]
    }

    except_cols = ['size_unknown_unit', 'app_category']
    group_cols = df.columns.difference(except_cols)
    duplicate_rows = df[df.duplicated(subset=group_cols, keep=False)]
    grouped = duplicate_rows.groupby(list(group_cols))

    resolved_rows = []
    skipped_indexes = []

    for _, group_df in grouped:
        size_values = pd.to_numeric(group_df['size_unknown_unit'], errors='coerce')
        size_mean = str(math.floor(size_values.mean())) if not size_values.isna().all() else None

        category_values = group_df['app_category'].dropna().unique()
        app_tags = str(group_df['app_tags'].iloc[0]) if 'app_tags' in group_df else ""
        app_tags_lower = app_tags.lower()

        ### Step 1: Try original logic — exact category in tags
        matched_direct = [cat for cat in category_values if cat.lower().replace('_', ' ') in app_tags_lower]

        if len(matched_direct) == 1:
            chosen_category = matched_direct[0]
        else:
            ### Step 2: Try using custom tag map
            matched_from_map = []
            for cat in category_values:
                if cat in category_tag_map:
                    for keyword in category_tag_map[cat]:
                        if keyword.lower() in app_tags_lower:
                            matched_from_map.append(cat)
                            break
            matched_from_map = list(set(matched_from_map))

            chosen_category = matched_from_map[0] if len(matched_from_map) == 1 else None

        if chosen_category:
            resolved_row = group_df.iloc[0].copy()
            resolved_row['size_unknown_unit'] = size_mean
            resolved_row['app_category'] = chosen_category
            resolved_rows.append(resolved_row)
        else:
            skipped_indexes.extend(group_df.index.tolist())

    # Drop only those that were resolved
    cleaned_df = df.drop(index=duplicate_rows.index.difference(skipped_indexes))
    cleaned_df = pd.concat([cleaned_df, pd.DataFrame(resolved_rows)], ignore_index=True)

    return cleaned_df


* This function will take the mean for the size
* For the category it will look if one of the categories match the tags, if so it will use that; if neither or both match it will not drop the column
* We also add custom mappings for vague tags

In [312]:
train_df_copy = deduplicate_with_mean_and_tag_based_category(train_df_copy)
near_duplicates = find_almost_near_duplicates(train_df_copy, except_cols=['size_unknown_unit', 'app_category'])
near_duplicates

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y


In [308]:
train_df_copy.to_csv("cleaned_train_df.csv", index=False)

### Handle Varies with device

In [313]:
train_df_copy.Y.isnull().sum()

np.int64(1458)

In [314]:
(train_df_copy['compatible_os_version'] == "Varies with device").sum()

np.int64(845)

In [315]:
(train_df_copy['app_version'] == "Varies with device").sum()

np.int64(906)

In [316]:
(train_df_copy['downloads_unstandardized'] == "Varies with device").sum()

np.int64(1047)

* Null Y: 1469 -> 1458
* OS varies: 1087 -> 845
* App version varies: 1172 -> 906
* downloads varies: 1358 -> 1047

* For now we can remove any varies with device in download and with a null Y
* Also remove any that has varies in all 3

In [322]:
dropped_rows_varies_downloads_null_y = train_df_copy[
    (train_df_copy['downloads_unstandardized'] == "Varies with device") &
    (train_df_copy['Y'].isnull())
]
dropped_rows_varies_downloads_null_y

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
1423,OMD Protocols,MEDICAL,0,Varies with device,10+,Free,0,Everyone,Medical,"July 27, 2018",1.0,Varies with device,NaN
1823,SD card backup,TOOLS,142,Varies with device,"1,000,000+",Free,0,Everyone,Tools,"March 27, 2017",Varies with device,Varies with device,NaN
2403,F-1 watchface by Delta,PERSONALIZATION,0,Varies with device,10+,Paid,$2.49,Everyone,Personalization,"August 23, 2017",Varies with device,Varies with device,NaN
2460,D+H Reaction Wall,GAME,0,Varies with device,1+,Paid,$0.99,Everyone,Arcade,"July 24, 2016",Varies with device,Varies with device,NaN
2671,Q Actions - Digital Assistant,PRODUCTIVITY,0,Varies with device,500+,Free,0,Everyone,Productivity,"July 20, 2018",Varies with device,Varies with device,NaN
3138,Elim AG,LIFESTYLE,7,Varies with device,100+,Free,0,Everyone,Lifestyle,"March 31, 2017",1.0,4.2 and up,NaN
3140,AG Subway Simulator Pro,FAMILY,0,Varies with device,"1,000+",Free,0,Everyone,Simulation,"July 27, 2018",Varies with device,Varies with device,NaN
3153,Alabama Ag Credit Ag Banking,FINANCE,0,Varies with device,100+,Free,0,Everyone,Finance,"February 9, 2018",5.3.4,4.4 and up,NaN
3171,Woodland AH,MEDICAL,0,Varies with device,50+,Free,0,Everyone,Medical,"February 2, 2018",300000.0.97,4.0.3 and up,NaN
3235,ai.Bot Box,PRODUCTIVITY,0,Varies with device,"10,000+",Free,0,Everyone,Productivity,"March 1, 2017",Varies with device,Varies with device,NaN


In [325]:
train_df_copy = train_df_copy.drop(index=dropped_rows_varies_downloads_null_y.index)

In [331]:
dropped_rows_varies_all= train_df_copy[
    (train_df_copy['downloads_unstandardized'] == "Varies with device") &
    (train_df_copy['app_version'] == "Varies with device") &
    (train_df_copy['compatible_os_version'] == "Varies with device")
]
dropped_rows_varies_all

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
31,Used Cars and Trucks for Sale,AUTO_AND_VEHICLES,17057,Varies with device,"1,000,000+",Free,0,Everyone,Auto & Vehicles,"July 30, 2018",Varies with device,Varies with device,4.6
44,Ulysse Speedometer,AUTO_AND_VEHICLES,40211,Varies with device,"5,000,000+",Free,0,Everyone,Auto & Vehicles,"July 30, 2018",Varies with device,Varies with device,4.3
45,REPUVE,AUTO_AND_VEHICLES,356,Varies with device,"100,000+",Free,0,Everyone,Auto & Vehicles,"May 25, 2018",Varies with device,Varies with device,3.9
58,CarMax – Cars for Sale: Search Used Car Inventory,AUTO_AND_VEHICLES,21777,Varies with device,"1,000,000+",Free,0,Everyone,Auto & Vehicles,"August 4, 2018",Varies with device,Varies with device,4.4
71,Selfie Camera,BEAUTY,17934,Varies with device,"1,000,000+",Free,0,Everyone,Beauty,"September 12, 2017",Varies with device,Varies with device,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8127,Duolingo: Learn Languages Free,EDUCATION,6290215,Varies with device,"100,000,000+",Free,0,Everyone,Education;Education,"August 1, 2018",Varies with device,Varies with device,4.7
8129,"Learn languages, grammar & vocabulary with Mem...",EDUCATION,1107911,Varies with device,"10,000,000+",Free,0,Everyone,Education,"August 2, 2018",Varies with device,Varies with device,4.7
8131,Credit Karma,FINANCE,706301,Varies with device,"10,000,000+",Free,0,Everyone,Finance,"July 31, 2018",Varies with device,Varies with device,4.7
8132,BeyondMenu Food Delivery,FOOD_AND_DRINK,51530,Varies with device,"1,000,000+",Free,0,Everyone,Food & Drink,"August 1, 2018",Varies with device,Varies with device,4.7


In [332]:
train_df_copy = train_df_copy.drop(index=dropped_rows_varies_all.index)

In [337]:
train_df_copy.Y.isnull().sum()

np.int64(1416)

In [338]:
train_df_copy.to_csv("cleaned_train_df2.csv", index=False)

In [1]:
import pandas as pd

train_df_copy = pd.read_csv("cleaned_train_df2.csv")

### Final Cleanup for now

In [2]:
train_df_copy.dropna(subset=['Y'], inplace=True)

In [3]:
train_df_copy

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,4.7
1,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4.3
2,Paper flowers instructions,ART_AND_DESIGN,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,4.4
3,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,3.8
4,Infinite Painter,ART_AND_DESIGN,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7392,TED,EDUCATION,181927,18M,"10,000,000+",Free,0,Everyone 10+,Education,"July 27, 2018",3.2.5,4.1 and up,4.6
7393,Choices: Stories You Play,GAME,807246,93M,"10,000,000+",Free,0,Teen,Simulation,"July 30, 2018",2.3.5,4.0.3 and up,4.6
7394,Toca Life: City,EDUCATION,31092,24M,"500,000+",Paid,$3.99,Everyone,Education;Pretend Play,"July 6, 2018",1.5-play,4.4 and up,4.7
7395,UNICORN - Color By Number & Pixel Art Coloring,ART_AND_DESIGN,8204,24M,"500,000+",Free,0,Everyone,Art & Design;Creativity,"August 2, 2018",1.0.9,4.4 and up,4.7


In [7]:
dropped_rows_varies_downloads = train_df_copy[train_df_copy['downloads_unstandardized'] == "Varies with device"]
dropped_rows_varies_downloads

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
22,Floor Plan Creator,ART_AND_DESIGN,36639,Varies with device,"5,000,000+",Free,0,Everyone,Art & Design,"July 14, 2018",Varies with device,2.3.3 and up,4.1
47,PDD-UA,AUTO_AND_VEHICLES,736,Varies with device,"100,000+",Free,0,Everyone,Auto & Vehicles,"July 29, 2018",2.9,2.3.3 and up,4.8
59,Fuelio: Gas log & costs,AUTO_AND_VEHICLES,65786,Varies with device,"1,000,000+",Free,0,Everyone,Auto & Vehicles,"August 2, 2018",Varies with device,4.0.3 and up,4.6
73,Girls Hairstyles,BEAUTY,3595,Varies with device,"500,000+",Free,0,Everyone,Beauty,"May 24, 2018",Varies with device,4.0 and up,4.1
93,Cool Reader,BOOKS_AND_REFERENCE,246315,Varies with device,"10,000,000+",Free,0,Everyone,Books & Reference,"July 17, 2015",Varies with device,1.5 and up,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7339,Galaxy Attack: Alien Shooter,GAME,506434,Varies with device,"10,000,000+",Free,0,Everyone,Arcade,"August 1, 2018",5.66,4.1 and up,4.6
7340,Hill Climb Racing 2,GAME,2750527,Varies with device,"100,000,000+",Free,0,Everyone,Racing,"August 2, 2018",1.17.2,4.2 and up,4.6
7365,Toy Blast,GAME,1889416,Varies with device,"50,000,000+",Free,0,Everyone,Puzzle,"July 23, 2018",5423,4.1 and up,4.7
7370,Bowmasters,GAME,1535468,Varies with device,"50,000,000+",Free,0,Teen,Action,"July 23, 2018",2.12.5,4.1 and up,4.7


In [9]:
train_df_copy.drop(index=dropped_rows_varies_downloads.index, inplace=True)
train_df_copy

,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,4.7
1,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4.3
2,Paper flowers instructions,ART_AND_DESIGN,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,4.4
3,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,3.8
4,Infinite Painter,ART_AND_DESIGN,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7392,TED,EDUCATION,181927,18M,"10,000,000+",Free,0,Everyone 10+,Education,"July 27, 2018",3.2.5,4.1 and up,4.6
7393,Choices: Stories You Play,GAME,807246,93M,"10,000,000+",Free,0,Teen,Simulation,"July 30, 2018",2.3.5,4.0.3 and up,4.6
7394,Toca Life: City,EDUCATION,31092,24M,"500,000+",Paid,$3.99,Everyone,Education;Pretend Play,"July 6, 2018",1.5-play,4.4 and up,4.7
7395,UNICORN - Color By Number & Pixel Art Coloring,ART_AND_DESIGN,8204,24M,"500,000+",Free,0,Everyone,Art & Design;Creativity,"August 2, 2018",1.0.9,4.4 and up,4.7


In [14]:
dropped_rows_varies_os = train_df_copy[train_df_copy['compatible_os_version'] == "Varies with device"]
train_df_copy.drop(index=dropped_rows_varies_os.index, inplace=True)

dropped_rows_varies_version = train_df_copy[train_df_copy['app_version'] == "Varies with device"]
train_df_copy.drop(index=dropped_rows_varies_version.index, inplace=True)

train_df_copy


,app_name,app_category,size_unknown_unit,downloads_unstandardized,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,4.7
1,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4.3
2,Paper flowers instructions,ART_AND_DESIGN,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,4.4
3,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,3.8
4,Infinite Painter,ART_AND_DESIGN,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7392,TED,EDUCATION,181927,18M,"10,000,000+",Free,0,Everyone 10+,Education,"July 27, 2018",3.2.5,4.1 and up,4.6
7393,Choices: Stories You Play,GAME,807246,93M,"10,000,000+",Free,0,Teen,Simulation,"July 30, 2018",2.3.5,4.0.3 and up,4.6
7394,Toca Life: City,EDUCATION,31092,24M,"500,000+",Paid,$3.99,Everyone,Education;Pretend Play,"July 6, 2018",1.5-play,4.4 and up,4.7
7395,UNICORN - Color By Number & Pixel Art Coloring,ART_AND_DESIGN,8204,24M,"500,000+",Free,0,Everyone,Art & Design;Creativity,"August 2, 2018",1.0.9,4.4 and up,4.7


#### Cleanup columns

In [15]:
train_df_copy['downloads_unstandardized'].value_counts()

downloads_unstandardized
12M     124
14M     119
13M     108
11M     106
15M     105
       ... 
847k      1
364k      1
467k      1
552k      1
619k      1
Name: count, Length: 366, dtype: int64

In [18]:
def standardize_to_millions(value):
    """
    Convert string like '1.5M', '600K', or '200' to a float in millions (M).
    """
    if isinstance(value, str):
        value = value.strip().upper().replace(",", "")
        if value.endswith("M"):
            return float(value[:-1])
        elif value.endswith("K"):
            return float(value[:-1]) / 1_000
        elif value.replace('.', '', 1).isdigit():
            return float(value) / 1_000_000
    elif isinstance(value, (int, float)):
        return value / 1_000_000
    return None  # or np.nan


In [ ]:
train_df_copy['downloads_unstandardized'] = train_df_copy['downloads_unstandardized'].apply(standardize_to_millions)
train_df_copy.rename(columns={
    'downloads_unstandardized': 'downloads_millions'
}, inplace=True)
train_df_copy

In [30]:
train_df_copy = train_df_copy.loc[:, ~train_df_copy.T.duplicated()]
train_df_copy

,app_name,app_category,size_unknown_unit,downloads_millions,reviews_count,free_paid,price_if_paid,age_rating,app_tags,release_date,app_version,compatible_os_version,Y
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,87510,8.7,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,4.7
1,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,967,2.8,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4.3
2,Paper flowers instructions,ART_AND_DESIGN,167,5.6,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,4.4
3,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,178,19.0,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,3.8
4,Infinite Painter,ART_AND_DESIGN,36815,29.0,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7392,TED,EDUCATION,181927,18.0,"10,000,000+",Free,0,Everyone 10+,Education,"July 27, 2018",3.2.5,4.1 and up,4.6
7393,Choices: Stories You Play,GAME,807246,93.0,"10,000,000+",Free,0,Teen,Simulation,"July 30, 2018",2.3.5,4.0.3 and up,4.6
7394,Toca Life: City,EDUCATION,31092,24.0,"500,000+",Paid,$3.99,Everyone,Education;Pretend Play,"July 6, 2018",1.5-play,4.4 and up,4.7
7395,UNICORN - Color By Number & Pixel Art Coloring,ART_AND_DESIGN,8204,24.0,"500,000+",Free,0,Everyone,Art & Design;Creativity,"August 2, 2018",1.0.9,4.4 and up,4.7


In [31]:
train_df_copy['reviews_count'].value_counts()

reviews_count
1,000,000+        940
10,000+           784
100,000+          783
10,000,000+       527
1,000+            525
500,000+          385
5,000,000+        369
50,000+           337
5,000+            328
100+              237
500+              152
50,000,000+        92
100,000,000+       84
10+                51
50+                45
500,000,000+        7
5+                  5
1+                  3
1,000,000,000+      2
Name: count, dtype: int64

### Separate numerical & Categorical

In [20]:
def separate_numerical_categorical(dataframe):
    numerical_dataframe = dataframe[[
        "size_unknown_unit", "downloads_unstandardized",
        "price_if_paid", "reviews_count", "Y"
    ]]
    categorical_dataframe = dataframe[[
        "app_category", "reviews_count",
        "free_paid", "age_rating"
    ]]
    return numerical_dataframe, categorical_dataframe

In [21]:
numerical_df, categorical_df = separate_numerical_categorical(train_df_copy)

In [23]:
numerical_df.head()

,app_name,size_bytes,downloads_millions,price_if_paid,Y
0,Coloring book moana,967,14M,0,3.9
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",87510,8.7M,0,4.7
2,Pixel Draw - Number Art Coloring Book,967,2.8M,0,4.3
3,Paper flowers instructions,167,5.6M,0,4.4
4,Smoke Effect Photo Maker - Smoke Editor,178,19M,0,3.8
